# Predicting Mortality Risk of ICU Patients Post-CABG

<span style="color:red;"><b>
MAIN DATA CLEANING FILE
</span></b>

## Importing and Cleaning Data

In [1]:
import pandas as pd
import os
from os.path import isfile, join
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import RandomizedSearchCV
from sklearn.base import BaseEstimator
import tensorflow as tf

#from tensorflow import keras

2024-11-22 10:48:50.737060: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-22 10:48:50.771900: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-22 10:48:50.788979: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-22 10:48:50.794032: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-22 10:48:50.807127: I tensorflow/core/platform/cpu_feature_guar

In [85]:
# Get the current Conda environment
conda_env = os.environ.get('CONDA_DEFAULT_ENV')

# Print the environment name
print(f"Current Conda environment: {conda_env}")

# Set working directory
os.chdir('/sfs/gpfs/tardis/home/krb3ym/Documents/MSDS/DS6050/final_project/Predicting-Mortality-Risk-of-ICU-Patients-Post-Coronary-Artery-Bypass-Graft-Surgery/')

Current Conda environment: base


**Importing main data files and concatenating them**

In [86]:
root = r'./data/datasets/core_cabg_valves'

# data = {file.replace('.csv', ''): pd.read_csv(root + '/' + file) for file in listdir(root)}
data = {file.replace('.csv', ''): pd.read_csv(join(root, file)) 
        for file in os.listdir(root) if isfile(join(root, file))}

In [87]:
# Creating patient data
admissions = data['admissions_cabg_valves'][['subject_id','hadm_id', 'insurance', 'race', 'deathtime', 'discharge_location']]
patient = data['patients_cabg_valves'][['subject_id', 'gender', 'anchor_age']]
surgeries = data['surgeries']

# Merging them together
df = pd.merge(surgeries, admissions, on = ['subject_id', 'hadm_id'], how = 'inner')
df = df.merge(patient, how='inner', on= 'subject_id')

# Creating chart events data & ICU data
chart_events = data['chart_events'][['subject_id', 'hadm_id', 'charttime', 'value', 'valueuom', 'label']].copy()
chart_events.rename(columns={'charttime': 'endtime'}, inplace=True)
icu_stay = data['icu_stay']

# Merging back with data
df = df.merge(chart_events, how = 'left', on = ['subject_id', 'hadm_id'])
df = df.merge(icu_stay, how = 'left', on = ['subject_id', 'hadm_id'])

# Ordering data by time & sorting
df['endtime'] = pd.to_datetime(df['endtime'])
df['intime'] = pd.to_datetime(df['intime'])
df = df.sort_values(by=['subject_id', 'hadm_id', 'stay_id', 'intime', 'endtime']).reset_index(drop=True)

# Rearranging columns 
df = df[['subject_id', 'hadm_id', 'stay_id', 'intime', 'gender',
         'anchor_age', 'insurance', 'race', 'deathtime', 
         'discharge_location', 'is_avr', 'is_cabg', 'is_mitral', 
         'gender', 'endtime', 'value', 'valueuom', 'label']]

# df.to_csv('model_data.csv', index=False)

In [91]:
df.nunique()

subject_id              7497
hadm_id                 7562
stay_id                 7993
intime                  7993
gender                     2
anchor_age                71
insurance                  3
race                      33
deathtime                119
discharge_location        10
is_avr                     2
is_cabg                    2
is_mitral                  2
gender                     2
endtime               676886
value                    998
valueuom                   5
label                      7
dtype: int64

In [92]:
df[['subject_id', 'discharge_location']].drop_duplicates().discharge_location.value_counts()

HOME HEALTH CARE                4326
SKILLED NURSING FACILITY        2161
REHAB                            481
CHRONIC/LONG TERM ACUTE CARE     267
HOME                             144
DIED                             117
ACUTE HOSPITAL                    11
OTHER FACILITY                     9
HOSPICE                            6
AGAINST ADVICE                     4
Name: discharge_location, dtype: int64

**We are not going to keep only one ICU admission per patient**

This does not simulate real life examples

For patients admitted to the ICU multiple times, we will only look at their first admission. This is to ensure that observations are all independent.

In [116]:
# Creating New Mortality Data
df['mortality'] = np.where(df['discharge_location']  == 'DIED', True, False)

In [123]:
df[['subject_id', 'mortality']].drop_duplicates().mortality.sum()

21

In [125]:
21/7000*100

0.3

In [15]:
# ICU stays
icu_data = pd.read_csv('./data/datasets/icu_stays_filtered.csv')

# Converting to time format
icu_data['intime'] = pd.to_datetime(icu_data['intime'])

icu_data['Unique Stay'] =  icu_data['subject_id'].astype(str) + icu_data['hadm_id'].astype(str)

In [16]:
# Looking at number of ICU readmissions in general
icu_data.groupby(['subject_id', 'hadm_id'])['stay_id'].nunique().value_counts()

1    6000
2     583
3      55
4      10
6       1
Name: stay_id, dtype: int64

In [17]:
# Keeping only 1 ICU admission per patient
icu_data = icu_data.sort_values('intime').groupby(['Unique Stay'], as_index=False).first()

In [18]:
icu_data.groupby(['subject_id', 'hadm_id'])['stay_id'].nunique().value_counts()

1    6649
Name: stay_id, dtype: int64

In [19]:
# Merging data back with original data
data['Unique Stay'] = data['Unique Stay'].astype(str)
data = pd.merge(icu_data[['Unique Stay', 'subject_id', 'stay_id']], data, on = 'Unique Stay', how = 'inner')

In [20]:
data.head()

,Unique Stay,subject_id,stay_id,insurance,race,marital_status,deathtime,discharge_location,gender,anchor_age,endtime,value,amount,amountuom,label,Died,sequence_num
0,1000201323581541,10002013,39060235,Medicare,OTHER,SINGLE,NaN,HOME HEALTH CARE,F,53,2160-05-18 10:26:00,NaN,3.3,mmol/L,Lactic Acid,False,1
1,1000201323581541,10002013,39060235,Medicare,OTHER,SINGLE,NaN,HOME HEALTH CARE,F,53,2160-05-18 10:26:00,NaN,421.0,mmHg,Arterial O2 pressure,False,2
2,1000201323581541,10002013,39060235,Medicare,OTHER,SINGLE,NaN,HOME HEALTH CARE,F,53,2160-05-18 11:23:00,NaN,2.8,mmol/L,Lactic Acid,False,3
3,1000201323581541,10002013,39060235,Medicare,OTHER,SINGLE,NaN,HOME HEALTH CARE,F,53,2160-05-18 12:20:00,NaN,3.1,mmol/L,Lactic Acid,False,4
4,1000201323581541,10002013,39060235,Medicare,OTHER,SINGLE,NaN,HOME HEALTH CARE,F,53,2160-05-18 12:20:00,NaN,384.0,mmHg,Arterial O2 pressure,False,5


In [21]:
data.subject_id.nunique()

5346

In [22]:
data.stay_id.nunique()

5361

In [23]:
data['Unique Stay'].nunique()

5361

**Importing Charlson Comorbidities Data**

In [24]:
# Importing data
charlson_df = pd.read_csv('./data/datasets/charlson_cabg.csv')
charlson_df['Unique Stay'] = charlson_df['subject_id'].astype(str) + charlson_df['hadm_id'].astype(str)

# Merging charlson data with main data
data = pd.merge(data, charlson_df.drop(['subject_id', 'hadm_id'], axis =1), on = 'Unique Stay', how = 'left')
data.rename(columns = {'charlson_comorbidity_index': 'charlson'}, inplace = True)

**Importing ventilation data**

In [25]:
data.shape

(1214970, 18)

In [26]:
vent_data = pd.read_csv('./data/datasets/cabg_ventilation.csv')

In [27]:
# Converting start and stop vent time to date/time format
vent_data['vent_starttime'] = pd.to_datetime(vent_data['starttime'])
vent_data['vent_endtime'] = pd.to_datetime(vent_data['endtime'])
# Filtering vent data to only include CABBG patients
patient_id = [row[0:8] for row in data['Unique Stay'].astype('str')]
# Filtering the ventilator data for CABG patients based on identified stay_ids
vent_data = vent_data[vent_data['subject_id'].astype('str').isin(patient_id)].drop(['starttime', 'endtime'], axis = 1)

In [28]:
vent_data.head()

,subject_id,stay_id,ventilation_status,vent_starttime,vent_endtime
0,17997568,36100181,HFNC,2148-09-26 08:00:00,2148-09-26 17:23:00
1,17557505,34278743,HFNC,2167-04-23 12:00:00,2167-04-24 07:00:00
2,17557505,34278743,HFNC,2167-04-24 10:00:00,2167-04-24 16:00:00
3,11799303,32117491,HFNC,2183-11-08 10:00:00,2183-11-08 12:00:00
4,17579017,36645193,HFNC,2137-08-30 20:00:00,2137-08-30 21:30:00


**Converting to long format**

In [29]:
def expand_ventilation_data_merge(vent_data):
    # Convert datetime columns to datetime type
    vent_data['vent_starttime'] = pd.to_datetime(vent_data['vent_starttime'])
    vent_data['vent_endtime'] = pd.to_datetime(vent_data['vent_endtime'])
    
    # Create time ranges for each unique stay
    time_ranges = []
    for (subject_id, stay_id), group in vent_data.groupby(['subject_id', 'stay_id']):
        times = pd.date_range(
            start=group['vent_starttime'].min(),
            end=group['vent_endtime'].max(),
            freq='H'
        )
        time_ranges.append(pd.DataFrame({
            'subject_id': subject_id,
            'stay_id': stay_id,
            'time': times
        }))
    
    # Combine all time ranges into a single DataFrame
    time_df = pd.concat(time_ranges, ignore_index=True)
    
    # Perform a merge to filter times within the vent_starttime and vent_endtime intervals
    result = (
        time_df.merge(vent_data, on=['subject_id', 'stay_id'], how='left')
        .query('vent_starttime <= time <= vent_endtime')
        .groupby(['subject_id', 'stay_id', 'time'])
        .first()
        .reset_index()
    ).drop(['vent_starttime', 'vent_endtime'], axis = 1)
    
    return result

# Assuming vent_data is your DataFrame with ventilation data
vent_data_long = expand_ventilation_data_merge(vent_data)


In [30]:
# Merging ventilator data with aggregated data!
vent_data_long = vent_data_long.rename(columns = {'time': 'time_bucket'})
vent_data_long['time_bucket'] = vent_data_long['time_bucket'].dt.round('H')

In [31]:
vent_data_long

,subject_id,stay_id,time_bucket,ventilation_status
0,10002013,39060235,2160-05-18 14:00:00,InvasiveVent
1,10002013,39060235,2160-05-18 15:00:00,InvasiveVent
2,10002013,39060235,2160-05-18 16:00:00,InvasiveVent
3,10002013,39060235,2160-05-18 17:00:00,InvasiveVent
4,10002013,39060235,2160-05-18 18:00:00,InvasiveVent
...,...,...,...,...
388074,19995790,34995866,2185-02-03 08:00:00,SupplementalOxygen
388075,19995790,34995866,2185-02-03 09:00:00,SupplementalOxygen
388076,19995790,34995866,2185-02-03 10:00:00,SupplementalOxygen
388077,19995790,34995866,2185-02-03 11:00:00,SupplementalOxygen


We don't need `ventilator type` and `ventilator mode` from the columns data.label anymore. So we can remove these rows are they are rendered useless with the new variable `ventilation_status`. This will also reduce the number of columns when we reshape the data from long to wide. 

In [32]:
# Deleting ventilator type and ventilator mode rows from dataset
data = data[~(data.label == 'Ventilator Type')]
data = data[~(data.label == 'Ventilator Mode')]

**Importing Vasoactive data**

In [33]:
vasoactive_data = pd.read_csv('./data/datasets/vasoactive_cabg.csv')

In [34]:
vasoactive_data.shape

(111538, 10)

In [35]:
# Merging with Unique Stay
vasoactive_data = pd.merge(vasoactive_data, icu_data[['stay_id', 'Unique Stay']], on = 'stay_id', how = 'inner')

# Converting start and stop vasoactive time to date/time format
vasoactive_data['vaso_starttime'] = pd.to_datetime(vasoactive_data['starttime'])
vasoactive_data['vaso_endtime'] = pd.to_datetime(vasoactive_data['endtime'])

# Filtering the vasoactive data for CABG patients based on identified stay_ids
vasoactive_data = vasoactive_data[vasoactive_data['stay_id'].astype('str').isin(data.stay_id.astype('str').to_list())].drop(['starttime', 'endtime'], axis = 1)

# Reordering columns
vasoactive_data = vasoactive_data[['stay_id', 'vaso_starttime', 'vaso_endtime', 'epinephrine', 'norepinephrine', 'phenylephrine', 'dobutamine', 'milrinone', 'dopamine']]

Converting vasoactive data it into an hourly time series, where each row represents one hour and shows the doses of all vasoactive medications (epinephrine, norepinephrine, phenylephrine, dobutamine, milrinone, and dopamine) that a patient was receiving during that hour.

In [36]:
def expand_vasoactive_data_merge(vasoactive_data):
    # Convert datetime columns
    vasoactive_data['vaso_starttime'] = pd.to_datetime(vasoactive_data['vaso_starttime'])
    vasoactive_data['vaso_endtime'] = pd.to_datetime(vasoactive_data['vaso_endtime'])
    
    # Create time ranges for each stay
    time_ranges = []
    for stay_id, group in vasoactive_data.groupby('stay_id'):
        times = pd.date_range(
            start=group['vaso_starttime'].min(),
            end=group['vaso_endtime'].max(),
            freq='H'
        )
        time_ranges.append(pd.DataFrame({
            'stay_id': stay_id,
            'time': times
        }))
    
    time_df = pd.concat(time_ranges, ignore_index=True)
    
    # Merge with original data using interval join logic
    result = time_df.merge(vasoactive_data, on='stay_id', how='left')
    
    # Filter for rows where time falls within the start and end times
    result = result[
        (result['time'] >= result['vaso_starttime']) & 
        (result['time'] < result['vaso_endtime'])
    ]
    
    # Get the latest dose for each medication at each hour
    vasoactive_agents = ['epinephrine', 'norepinephrine', 'phenylephrine', 
                        'dobutamine', 'milrinone', 'dopamine']
    
    # Group by stay_id and time, and get the last non-null value for each medication
    result = result.groupby(['stay_id', 'time'])[vasoactive_agents].last().reset_index()
    
    # Fill NaN values with 0 to indicate no medication at that time
    result[vasoactive_agents] = result[vasoactive_agents].fillna(0)
    
    return result

# Create the dataset
vasoactive_long = expand_vasoactive_data_merge(vasoactive_data)

# Merging ventilator data with aggregated data!
vasoactive_long = vasoactive_long.rename(columns = {'time': 'time_bucket'})
vasoactive_long['time_bucket'] = vasoactive_long['time_bucket'].dt.floor('H')

In [37]:
# Deleting vasoactive data from label variable since these will be superceded by the vasoactive_long table
data = data[~data.label.isin(['Norepinephrine', 'Dobutamine', 'Dopamine', 'Epinephrine', 'Epinephrine.'])]

In [38]:
data.label.value_counts()

Heart Rate                      526542
Arterial Blood Pressure mean    336641
Inspired O2 Fraction             71942
Arterial O2 pressure             61075
Platelet Count                   36745
Creatinine (serum)               34050
Lactic Acid                      31799
Total Bilirubin                   4365
Name: label, dtype: int64

**Merging vasoactive with ventilator data**  

This will create on long format table, where for each hour, and for each patient, we have information regarding what drips they were on and what were the vent settings

In [39]:
# Merging ventilator with vasoactive data
rx_data_long = pd.merge(vasoactive_long, vent_data_long, on=['stay_id', 'time_bucket'], how='outer', suffixes=('', '_vaso'))


# Dropping subject_id column
rx_data_long = rx_data_long.drop('subject_id', axis = 1)

We will replace missing values for drips with 0 - we assume that if patients did not have any drips recorded at that time, that they were not receiving anything. 

Same goes for ventilation status: if there is no documented ventilation/oxygenation treatment, we assume that patients were not receiving treatment and thus we set the value to be `none`. But we will do that later, because we need ventilation status to impute missing fio2 status

In [40]:
# Replace missing values with 0
rx_data_long[['epinephrine', 'norepinephrine', 'phenylephrine', 'dobutamine', 'dopamine', 'milrinone']] = rx_data_long[['epinephrine', 'norepinephrine', 'phenylephrine', 'dobutamine', 'dopamine', 'milrinone']].fillna(0)

# Rounding to 3 digits
rx_data_long[['epinephrine', 'norepinephrine', 'phenylephrine', 'dobutamine', 'dopamine', 'milrinone']] = rx_data_long[['epinephrine', 'norepinephrine', 'phenylephrine', 'dobutamine', 'dopamine', 'milrinone']].round(3)

In [41]:
rx_data_long.stay_id.nunique()

6791

In [42]:
rx_data_long['stay_id'].nunique()

6791

In [43]:
rx_data_long.head()

,stay_id,time_bucket,epinephrine,norepinephrine,phenylephrine,dobutamine,milrinone,dopamine,ventilation_status
0,30001148,2156-08-30 14:00:00,0.0,0.0,0.4,0.0,0.0,0.0,NaN
1,30001148,2156-08-30 15:00:00,0.0,0.0,0.0,0.0,0.0,0.0,InvasiveVent
2,30001148,2156-08-30 16:00:00,0.0,0.0,0.0,0.0,0.0,0.0,InvasiveVent
3,30001148,2156-08-30 17:00:00,0.0,0.0,0.0,0.0,0.0,0.0,InvasiveVent
4,30001148,2156-08-30 18:00:00,0.0,0.0,0.5,0.0,0.0,0.0,InvasiveVent


### Data Preprocessing

**Checking accuracy in coding of the variable `Died`**. 

Many patients who were coded as 'Died' were in fact discharged from the hospital. This is because this variable records whether patients died, even after discharge from the hospital. We are only interested in in-hospital mortality, so we need to create a new variable for that, which we will call `mortality`

In [44]:
data[data.Died == True].discharge_location.value_counts()

DIED                            75521
CHRONIC/LONG TERM ACUTE CARE    70267
SKILLED NURSING FACILITY        51304
REHAB                           32346
HOME HEALTH CARE                26007
HOSPICE                          6374
HOME                              363
OTHER FACILITY                    281
Name: discharge_location, dtype: int64

In [45]:
# Creating New Mortality Data
data['mortality'] = np.where(data['discharge_location']  == 'DIED', True, False)

In [46]:
# Dropping variable Died and discharge_location
data.drop('Died', axis = 1, inplace = True)
data.drop('discharge_location', axis = 1, inplace = True)

**Creating variable that codes for TIME OF DEATH, as a time-to-event outcome**

I will do that later - for now, we're interested in looking at patterns that predict in-hospital mortality in general. It's a classification problem. Later, we will build a survival model so that we can predict time-to-mortality.

In [47]:
# Recoding variable death time
data['deathtime'] = pd.to_datetime(data['deathtime'])

In [48]:
data[~data.deathtime.isna()].head()

,Unique Stay,subject_id,stay_id,insurance,race,marital_status,deathtime,gender,anchor_age,endtime,value,amount,amountuom,label,sequence_num,charlson,mortality
8726,1010445023157316,10104450,31524085,Medicare,WHITE,MARRIED,2137-10-13 00:01:00,M,82,2137-10-10 06:20:00,NaN,203.0,K/uL,Platelet Count,1,13,True
8727,1010445023157316,10104450,31524085,Medicare,WHITE,MARRIED,2137-10-13 00:01:00,M,82,2137-10-10 06:20:00,NaN,2.9,mg/dL,Creatinine (serum),2,13,True
8728,1010445023157316,10104450,31524085,Medicare,WHITE,MARRIED,2137-10-13 00:01:00,M,82,2137-10-10 07:46:00,NaN,99.0,bpm,Heart Rate,3,13,True
8729,1010445023157316,10104450,31524085,Medicare,WHITE,MARRIED,2137-10-13 00:01:00,M,82,2137-10-10 07:48:00,NaN,94.0,bpm,Heart Rate,4,13,True
8730,1010445023157316,10104450,31524085,Medicare,WHITE,MARRIED,2137-10-13 00:01:00,M,82,2137-10-10 07:48:00,NaN,94.0,bpm,Heart Rate,5,13,True


In [49]:
# Making sure that all patients who died have a time of death recorded (should return True)
data[~data.deathtime.isna()].mortality.value_counts()[0] == data[~data.deathtime.isna()].shape[0]

True

In [ ]:
data['mortality_time']  

<span style = 'color:maroon'> <b> ... to be continued later </span> </b>

**Recoding `race` data**

In [153]:
# Recoding race data
data['race_new']  = data.race.replace({
                              'WHITE': 'White',
                              'WHITE - OTHER EUROPEAN': 'White',
                              'WHITE - RUSSIAN': 'White',
                              'WHITE - EASTERN EUROPEAN': 'White',
                              'PORTUGUESE': 'White',
                              'WHITE - BRAZILIAN': 'White',
                              # Black
                              'BLACK/AFRICAN AMERICAN': 'Black',
                              'BLACK/AFRICAN': 'Black',
                              'HISPANIC/LATINO - GUATEMALAN': 'Black',
                              # Asian
                              'ASIAN - ASIAN INDIAN': 'Asian',
                              'ASIAN': 'Asian',
                              'ASIAN - CHINESE': 'Asian',
                              'ASIAN - SOUTH EAST ASIAN': 'Asian',
                              'ASIAN - KOREAN': 'Asian',
                              # Hispanic
                              'HISPANIC/LATINO - PUERTO RICAN': 'Hispanic',
                              'HISPANIC/LATINO - DOMINICAN': 'Hispanic',
                              'HISPANIC OR LATINO': 'Hispanic',
                              'HISPANIC': 'Hispanic',
                              'HISPANIC/LATINO - CUBAN': 'Hispanic',
                              'HISPANIC/LATINO - SALVADORAN': 'Hispanic',
                              'SOUTH AMERICAN': 'Hispanic',
                              'HISPANIC/LATINO - COLUMBIAN': 'Hispanic',
                              'HISPANIC/LATINO - CUBAN': 'Hispanic',
                              'HISPANIC/LATINO - HONDURAN': 'Hispanic',
                              'HISPANIC/LATINO - CENTRAL AMERICAN': 'Hispanic',
                              'HISPANIC/LATINO - MEXICAN': 'Hispanic',
                              # Native American/Pacific Islander
                              'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER': 'Native American',
                              'AMERICAN INDIAN/ALASKA NATIVE': 'Native American',
                              'BLACK/CAPE VERDEAN': 'Black',
                              'BLACK/CARIBBEAN ISLAND' : 'Black',
                              # Other
                              'OTHER': 'Other',
                              'MULTIPLE RACE/ETHNICITY': 'Other',
                              # Unknown
                              'UNABLE TO OBTAIN': np.nan,
                              'UNKNOWN': np.nan,
                              'PATIENT DECLINED TO ANSWER': np.nan,
                              })

data.race = data.race_new
data.drop(['race_new'], axis = 1, inplace = True)

data.race.value_counts()

White              790246
Black               45499
Hispanic            31957
Other               30206
Asian               22902
Native American      6440
Name: race, dtype: int64

In [154]:
# Checking for discrepancies in the race data

# Creating data for patient ID
data['subject_id'] = data['Unique Stay'].astype(str).str[0:8]

# Checking if all race data is the same for each patient
data.groupby('subject_id').race.nunique().value_counts()

1    4533
0     813
Name: race, dtype: int64

In [155]:
# **Cleaning up the group of variables `value`, `amount`, `amountuom`, `label`.**

In [156]:
data.label.value_counts()

Heart Rate                      526542
Arterial Blood Pressure mean    336641
Inspired O2 Fraction             71942
Arterial O2 pressure             61075
Platelet Count                   36745
Creatinine (serum)               34050
Lactic Acid                      31799
Total Bilirubin                   4365
Name: label, dtype: int64

**Pivoting Data**

In [157]:
data.head()

,Unique Stay,subject_id,stay_id,insurance,race,marital_status,deathtime,gender,anchor_age,endtime,value,amount,amountuom,label,sequence_num,charlson,mortality
0,1000201323581541,10002013,39060235,Medicare,Other,SINGLE,NaT,F,53,2160-05-18 10:26:00,NaN,3.3,mmol/L,Lactic Acid,1,7,False
1,1000201323581541,10002013,39060235,Medicare,Other,SINGLE,NaT,F,53,2160-05-18 10:26:00,NaN,421.0,mmHg,Arterial O2 pressure,2,7,False
2,1000201323581541,10002013,39060235,Medicare,Other,SINGLE,NaT,F,53,2160-05-18 11:23:00,NaN,2.8,mmol/L,Lactic Acid,3,7,False
3,1000201323581541,10002013,39060235,Medicare,Other,SINGLE,NaT,F,53,2160-05-18 12:20:00,NaN,3.1,mmol/L,Lactic Acid,4,7,False
4,1000201323581541,10002013,39060235,Medicare,Other,SINGLE,NaT,F,53,2160-05-18 12:20:00,NaN,384.0,mmHg,Arterial O2 pressure,5,7,False


In [184]:
tod = data[['Unique Stay', 'stay_id', 'deathtime']].drop_duplicates() 

# Pivot data from long to wide
data_wide = data.pivot_table(
    index=['insurance', 'race', 'endtime', 'marital_status', 'gender', 'anchor_age', 'mortality', 'Unique Stay', 'stay_id', 'sequence_num', 'charlson'],
    columns='label',
    values='amount',
    aggfunc= 'first'
).reset_index()

# Merging back with time of death
data_wide = pd.merge(data_wide, tod, on = ['Unique Stay', 'stay_id'], how = 'left')

In [186]:
pd.set_option('display.max_columns', None)
data_wide.head(5)

,insurance,race,endtime,marital_status,gender,anchor_age,mortality,Unique Stay,stay_id,sequence_num,charlson,Arterial Blood Pressure mean,Arterial O2 pressure,Creatinine (serum),Heart Rate,Inspired O2 Fraction,Lactic Acid,Platelet Count,Total Bilirubin,deathtime
0,Medicaid,Asian,2113-06-29 16:57:00,MARRIED,M,69,False,1356278426556587,30830290,1,4,NaN,332.0,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,Medicaid,Asian,2113-06-29 16:57:00,MARRIED,M,69,False,1356278426556587,30830290,2,4,NaN,NaN,NaN,NaN,NaN,4.6,NaN,NaN,NaT
2,Medicaid,Asian,2113-06-29 16:57:00,MARRIED,M,69,False,1356278426556587,30830290,3,4,NaN,NaN,NaN,NaN,NaN,4.6,NaN,NaN,NaT
3,Medicaid,Asian,2113-06-29 18:00:00,MARRIED,M,69,False,1356278426556587,30830290,5,4,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaT
4,Medicaid,Asian,2113-06-29 18:00:00,MARRIED,M,69,False,1356278426556587,30830290,6,4,NaN,NaN,NaN,NaN,50.0,NaN,NaN,NaN,NaT


<b>For each patient, need to create buckets of 1 hour, and then project values of all the measurements for that hour</b>

In [187]:
# Making sure endtime is in correct date time format
data_wide['endtime'] = pd.to_datetime(data_wide['endtime'])

# Create a new column with the rounded down hourly bucket
data_wide['time_bucket'] = data_wide['endtime'].dt.floor('H')

In [188]:
# Sample aggregation functions
agg_funcs = {
    'stay_id': 'first', 
    'anchor_age': 'first', 
    'insurance': 'first', 
    'race': 'first',
    'charlson': 'first',
    'marital_status': 'first',
    'gender': 'first',
    'mortality': 'max',
    'deathtime': lambda x: x.min() if x.notna().any() else np.nan,  # Keep NaN if all values are missing
    'sequence_num': 'min',
    'Arterial Blood Pressure mean': 'mean',
    'Arterial O2 pressure': 'mean',
    'Creatinine (serum)': 'mean',
    'Heart Rate': 'mean',
    'Inspired O2 Fraction': 'mean',
    'Lactic Acid': 'mean',
    'Platelet Count': 'mean',
}

In [189]:
# Sort data by 'Unique Stay' and 'time_bucket'
data_wide = data_wide.sort_values(by=['Unique Stay', 'time_bucket'])

# Group by 'Unique Stay' and 'time_bucket' and apply aggregation
data_wide_agg = data_wide.drop('endtime', axis=1).groupby(['Unique Stay', 'time_bucket']).agg(agg_funcs).reset_index()
data_wide_agg.head(7)

,Unique Stay,time_bucket,stay_id,anchor_age,insurance,race,charlson,marital_status,gender,mortality,deathtime,sequence_num,Arterial Blood Pressure mean,Arterial O2 pressure,Creatinine (serum),Heart Rate,Inspired O2 Fraction,Lactic Acid,Platelet Count
0,1000201323581541,2160-05-18 10:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,1,NaN,421.0,NaN,NaN,NaN,3.3,NaN
1,1000201323581541,2160-05-18 11:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,3,NaN,NaN,NaN,NaN,NaN,2.8,NaN
2,1000201323581541,2160-05-18 12:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,4,NaN,384.0,NaN,NaN,NaN,3.1,NaN
3,1000201323581541,2160-05-18 13:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,7,NaN,311.0,NaN,NaN,NaN,2.6,252.0
4,1000201323581541,2160-05-18 14:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,11,94.0,332.0,1.1,80.0,100.0,NaN,254.0
5,1000201323581541,2160-05-18 15:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,21,66.0,NaN,NaN,NaN,50.0,NaN,NaN
6,1000201323581541,2160-05-18 16:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,27,NaN,NaN,NaN,92.0,NaN,NaN,NaN


**I only kept the first ICU admission for each patient that has multiple ICU admission**

In [190]:
data_wide_agg.stay_id.nunique()

4451

In [191]:
data_wide_agg['Unique Stay'].nunique()

4451

In [192]:
# Creating index for grouping patients
group_idx = ['Unique Stay', 'time_bucket']

In [193]:
data_wide_agg.shape

(313940, 19)

**Merging data with ventilator and vasoactive data**

In [194]:
rx_data_long.shape

(411096, 9)

In [195]:
data_merged = pd.merge(data_wide_agg.drop(['Unique Stay'], axis = 1), rx_data_long, on = ['stay_id', 'time_bucket'], how = 'left')

In [196]:
data_merged['stay_id'].nunique()

4451

In [197]:
data_merged['ventilation_status'].value_counts()

SupplementalOxygen    118242
InvasiveVent           76386
HFNC                    2422
Tracheostomy            2295
NonInvasiveVent         1585
None                      60
Name: ventilation_status, dtype: int64

In [198]:
# Imputing missing ventilation status as 'None' (see explanation above)
data_merged['ventilation_status'] = data_merged['ventilation_status'].fillna('None')

In [199]:
# Reconding ventilation into yes/no binary variable
data_merged['vent'] = data_merged['ventilation_status'].apply(lambda x: 1 if x in ['InvasiveVent', 'Tracheostomy'] else 0)

# Will drop the old ventilation_status variable later

In [200]:
data_merged

,time_bucket,stay_id,anchor_age,insurance,race,charlson,marital_status,gender,mortality,deathtime,sequence_num,Arterial Blood Pressure mean,Arterial O2 pressure,Creatinine (serum),Heart Rate,Inspired O2 Fraction,Lactic Acid,Platelet Count,epinephrine,norepinephrine,phenylephrine,dobutamine,milrinone,dopamine,ventilation_status,vent
0,2160-05-18 10:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,1,NaN,421.0,NaN,NaN,NaN,3.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0
1,2160-05-18 11:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,3,NaN,NaN,NaN,NaN,NaN,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0
2,2160-05-18 12:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,4,NaN,384.0,NaN,NaN,NaN,3.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0
3,2160-05-18 13:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,7,NaN,311.0,NaN,NaN,NaN,2.6,252.0,NaN,NaN,NaN,NaN,NaN,NaN,None,0
4,2160-05-18 14:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,11,94.0,332.0,1.1,80.0,100.0,NaN,254.0,0.0,0.0,0.701,0.0,0.0,0.0,InvasiveVent,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319026,2185-02-03 10:00:00,34995866,66,Medicare,White,3,DIVORCED,M,False,NaT,76,NaN,NaN,NaN,104.0,NaN,NaN,NaN,0.0,0.0,0.000,0.0,0.0,0.0,SupplementalOxygen,0
319027,2185-02-03 11:00:00,34995866,66,Medicare,White,3,DIVORCED,M,False,NaT,78,NaN,NaN,NaN,82.0,NaN,NaN,NaN,0.0,0.0,0.000,0.0,0.0,0.0,SupplementalOxygen,0
319028,2185-02-03 12:00:00,34995866,66,Medicare,White,3,DIVORCED,M,False,NaT,81,NaN,NaN,NaN,85.0,NaN,NaN,NaN,0.0,0.0,0.000,0.0,0.0,0.0,SupplementalOxygen,0
319029,2185-02-03 13:00:00,34995866,66,Medicare,White,3,DIVORCED,M,False,NaT,83,NaN,NaN,NaN,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0


**Forward/Backward filling missing values for _FIXED_ variables**

In [201]:
# List of columns to forward fill
columns_to_fill = ['anchor_age', 'insurance', 'race', 'marital_status', 'gender', 'mortality']

# Forward fill specific columns within each group
data_merged[columns_to_fill] = data_merged.groupby('stay_id')[columns_to_fill].ffill().bfill()

In [202]:
# Recoding Sequence_number variable
data_merged['seq_num'] = data_merged.groupby(['stay_id']).cumcount()+1

data_merged.drop('sequence_num', axis = 1, inplace = True)

In [203]:
data_merged.head()

,time_bucket,stay_id,anchor_age,insurance,race,charlson,marital_status,gender,mortality,deathtime,Arterial Blood Pressure mean,Arterial O2 pressure,Creatinine (serum),Heart Rate,Inspired O2 Fraction,Lactic Acid,Platelet Count,epinephrine,norepinephrine,phenylephrine,dobutamine,milrinone,dopamine,ventilation_status,vent,seq_num
0,2160-05-18 10:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,NaN,421.0,NaN,NaN,NaN,3.3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0,1
1,2160-05-18 11:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,NaN,NaN,NaN,NaN,NaN,2.8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0,2
2,2160-05-18 12:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,NaN,384.0,NaN,NaN,NaN,3.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,0,3
3,2160-05-18 13:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,NaN,311.0,NaN,NaN,NaN,2.6,252.0,NaN,NaN,NaN,NaN,NaN,NaN,None,0,4
4,2160-05-18 14:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,94.0,332.0,1.1,80.0,100.0,NaN,254.0,0.0,0.0,0.701,0.0,0.0,0.0,InvasiveVent,1,5


In [204]:
# Renaming columns => easier to work with lower case names
data_merged = data_merged.rename(columns={'Arterial Blood Pressure mean': 'map',
                                      'Arterial O2 pressure': 'pao2',
                                      'Creatinine (serum)': 'creatinine',
                                      'Heart Rate': 'hr',
                                      'Inspired O2 Fraction': 'fio2',
                                      'Lactic Acid': 'lactate',
                                      'Platelet Count': 'platelets'})

**Doing the same thing as before: filling missing treatment observations with 0**

In [205]:
data_merged[['epinephrine', 'norepinephrine', 'phenylephrine', 'dobutamine', 'dopamine', 'milrinone', 'vent']] = data_merged[['epinephrine', 'norepinephrine', 'phenylephrine', 'dobutamine', 'dopamine', 'milrinone', 'vent']].fillna(0)

In [206]:
data_merged.head()

,time_bucket,stay_id,anchor_age,insurance,race,charlson,marital_status,gender,mortality,deathtime,map,pao2,creatinine,hr,fio2,lactate,platelets,epinephrine,norepinephrine,phenylephrine,dobutamine,milrinone,dopamine,ventilation_status,vent,seq_num
0,2160-05-18 10:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,NaN,421.0,NaN,NaN,NaN,3.3,NaN,0.0,0.0,0.000,0.0,0.0,0.0,None,0,1
1,2160-05-18 11:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,NaN,NaN,NaN,NaN,NaN,2.8,NaN,0.0,0.0,0.000,0.0,0.0,0.0,None,0,2
2,2160-05-18 12:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,NaN,384.0,NaN,NaN,NaN,3.1,NaN,0.0,0.0,0.000,0.0,0.0,0.0,None,0,3
3,2160-05-18 13:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,NaN,311.0,NaN,NaN,NaN,2.6,252.0,0.0,0.0,0.000,0.0,0.0,0.0,None,0,4
4,2160-05-18 14:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,94.0,332.0,1.1,80.0,100.0,NaN,254.0,0.0,0.0,0.701,0.0,0.0,0.0,InvasiveVent,1,5


**Merging data with OXYGEN DELIVERY FIO2 data from mimic IV derived table** 

In [207]:
# Importing FiO2 data from Mimic IV derived data - this specifically includes the Fio2 for when patients were on the vent. 
o2_delivery = pd.read_csv('./data/datasets/oxygen_delivery_cabg.csv')
o2_delivery['time_bucket'] = pd.to_datetime(o2_delivery['charttime']).dt.floor('H') # create 1hour time buckets
o2_delivery.drop('charttime', axis = 1, inplace = True)

# Keeping only rows where there is an oxygen delivery device 
o2_delivery = o2_delivery[~ (o2_delivery['o2_delivery_device_1'] == 'None')]
o2_delivery = o2_delivery[~(o2_delivery.o2_delivery_device_1.isna())]

# Dropping endotrachel tube
o2_delivery = o2_delivery[~ (o2_delivery['o2_delivery_device_1'] == 'Endotracheal tube')]

# Defining maximal flow rates for each device
max_flow_rates = {
    'Nasal cannula': 6,
    'Aerosol-cool': 15,
    'Face tent': 15,
    'High flow nasal cannula': 60,
    'Trach mask': 15,
    'High flow neb': 15,
    'CPAP mask': 60,
    'Bipap mask': 60,
    'Non-rebreather': 15,
    'Oxymizer': 15,
    'Medium conc mask': 10,
    'Other': np.nan,  # Variable; keep as NaN or assign a default if known
    'Venti mask': 15,
    'T-piece': 15,
    'Tracheostomy tube': 60,
    'Ultrasonic neb': 15}

# Capping flow rates with values capped by max flow rates
o2_delivery['o2_flow'] = o2_delivery.apply(lambda row: np.nan if row['o2_flow'] > max_flow_rates.get(row['o2_delivery_device_1'], np.nan) else row['o2_flow'], axis=1)
o2_delivery['o2_flow_additional'] = o2_delivery.apply(lambda row: np.nan if row['o2_flow_additional'] > max_flow_rates.get(row['o2_delivery_device_1'], np.nan) else row['o2_flow_additional'], axis=1)

# Imputing Fio2 based on flow rate and oxygen delivery device. Based on this paper: https://static-content.springer.com/esm/art%3A10.1038%2Fs41598-019-38491-0/MediaObjects/41598_2019_38491_MOESM1_ESM.pdf
def calculate_fio2(row):
    # Use the maximum of the two flow rates, or one if the other is NaN
    o2_flow = row['o2_flow'] if not np.isnan(row['o2_flow']) else 0
    o2_flow_additional = row['o2_flow_additional'] if not np.isnan(row['o2_flow_additional']) else 0
    max_flow = max(o2_flow, o2_flow_additional)
    
    # Apply FiO2 formulas based on device type
    device = row['o2_delivery_device_1']
    if device == 'Aerosol-cool':
        return min(60, 21 + (max_flow * 4))
    if device == 'Face tent':
        return min(40, 21 + (max_flow * 4) if max_flow > 0 else 25)
    elif device == 'Nasal cannula':
        return min(40, 21 + (max_flow * 4))
    elif device == 'High flow nasal cannula':
        return min(100, 48 + ((max_flow - 6) * 2) if max_flow >= 6 else 48)
    elif device == 'CPAP mask':
        return 40
    elif device == 'BIPAP mask':
        return 40    
    elif device == 'Non-rebreather':
        return min(100, 80 + (min((max_flow - 10), 2) * 10))
    elif device == 'Venti mask':
        return min(55, 26 + ((max_flow - 4) * 2.5) if max_flow >= 4 else 26)
    elif device == 'Oxymizer':
        return 40  # Default FiO2 for Oxymizer
    elif device == 'T-piece':
        return 40  # Default FiO2 for Trach mask
    elif device == 'Tracheostomy tube':
        return 40  # Default FiO2 for Trach mask
    else:
        return np.nan  # Return NaN if device type is unknown or not specified

# Apply the function to calculate FiO2
o2_delivery['fio2'] = o2_delivery.apply(calculate_fio2, axis=1)

o2_delivery.drop(['subject_id', 'o2_flow', 'o2_flow_additional'], axis = 1 ,inplace = True)
o2_delivery.sort_values(by=['stay_id','time_bucket'], inplace = True)

# Group by 'stay_id' and 'time_bucket' and apply aggregation
o2_delivery = o2_delivery.groupby(['stay_id', 'time_bucket']).agg({'fio2': 'mean', 'o2_delivery_device_1': 'first'}).reset_index()

# Dropping missing rows
o2_delivery = o2_delivery[~o2_delivery.fio2.isna()]

o2_delivery

,stay_id,time_bucket,fio2,o2_delivery_device_1
0,30001148,2156-08-30 18:00:00,40.0,Face tent
1,30001148,2156-08-30 20:00:00,40.0,Face tent
2,30001148,2156-08-31 00:00:00,37.0,Nasal cannula
3,30001148,2156-08-31 04:00:00,29.0,Nasal cannula
4,30004530,2165-08-01 05:00:00,60.0,Aerosol-cool
...,...,...,...,...
74601,39995735,2124-08-21 16:00:00,29.0,Nasal cannula
74602,39995735,2124-08-21 20:00:00,29.0,Nasal cannula
74603,39995735,2124-08-22 00:00:00,33.0,Nasal cannula
74604,39995735,2124-08-22 04:00:00,33.0,Nasal cannula


In [208]:
# Merging dataset
data_merged = pd.merge(data_merged, o2_delivery, on = ['stay_id', 'time_bucket'], how = 'left', suffixes = ('_original', '_delivery'))

# Replace missing values in 'fio2_original' with 'fio2_delivery' if 'fio2_delivery' is not missing
data_merged['fio2'] = np.where(data_merged['fio2_original'].isna() & ~data_merged['fio2_delivery'].isna(), data_merged['fio2_delivery'], data_merged['fio2_original'])

# Dropping columns
data_merged.drop(['fio2_delivery', 'fio2_original'], axis = 1, inplace = True)

In [209]:
data_merged

,time_bucket,stay_id,anchor_age,insurance,race,charlson,marital_status,gender,mortality,deathtime,map,pao2,creatinine,hr,lactate,platelets,epinephrine,norepinephrine,phenylephrine,dobutamine,milrinone,dopamine,ventilation_status,vent,seq_num,o2_delivery_device_1,fio2
0,2160-05-18 10:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,NaN,421.0,NaN,NaN,3.3,NaN,0.0,0.0,0.000,0.0,0.0,0.0,None,0,1,NaN,NaN
1,2160-05-18 11:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,NaN,NaN,NaN,NaN,2.8,NaN,0.0,0.0,0.000,0.0,0.0,0.0,None,0,2,NaN,NaN
2,2160-05-18 12:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,NaN,384.0,NaN,NaN,3.1,NaN,0.0,0.0,0.000,0.0,0.0,0.0,None,0,3,NaN,NaN
3,2160-05-18 13:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,NaN,311.0,NaN,NaN,2.6,252.0,0.0,0.0,0.000,0.0,0.0,0.0,None,0,4,NaN,NaN
4,2160-05-18 14:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,94.0,332.0,1.1,80.0,NaN,254.0,0.0,0.0,0.701,0.0,0.0,0.0,InvasiveVent,1,5,NaN,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319026,2185-02-03 10:00:00,34995866,66,Medicare,White,3,DIVORCED,M,False,NaT,NaN,NaN,NaN,104.0,NaN,NaN,0.0,0.0,0.000,0.0,0.0,0.0,SupplementalOxygen,0,20,NaN,NaN
319027,2185-02-03 11:00:00,34995866,66,Medicare,White,3,DIVORCED,M,False,NaT,NaN,NaN,NaN,82.0,NaN,NaN,0.0,0.0,0.000,0.0,0.0,0.0,SupplementalOxygen,0,21,NaN,NaN
319028,2185-02-03 12:00:00,34995866,66,Medicare,White,3,DIVORCED,M,False,NaT,NaN,NaN,NaN,85.0,NaN,NaN,0.0,0.0,0.000,0.0,0.0,0.0,SupplementalOxygen,0,22,Nasal cannula,33.0
319029,2185-02-03 13:00:00,34995866,66,Medicare,White,3,DIVORCED,M,False,NaT,NaN,NaN,NaN,79.0,NaN,NaN,0.0,0.0,0.000,0.0,0.0,0.0,None,0,23,NaN,NaN


**Merging data with ventilator FiO2 data**

In [210]:
# Importing FiO2 data from Mimic IV derived data - this specifically includes the Fio2 for when patients were on the vent. 
fio2 = pd.read_csv('./data/datasets/fio2_vent.csv')
fio2['time_bucket'] = pd.to_datetime(fio2['charttime']).dt.floor('H') # create 1hour time buckets
fio2.drop(['subject_id', 'flow_rate','ventilator_mode', 'ventilator_mode_hamilton','ventilator_type', 'charttime'], axis = 1 ,inplace = True)
fio2.sort_values(by=['stay_id','time_bucket'], inplace = True)

# Group by 'stay_id' and 'time_bucket' and apply aggregation
fio2 = fio2.groupby(['stay_id', 'time_bucket']).mean().reset_index()

# Dropping missing rows
fio2 = fio2[~fio2.fio2.isna()]

In [211]:
# Merging
data_merged = pd.merge(data_merged, fio2, on = ['stay_id', 'time_bucket'], how = 'left', suffixes=('_original', '_vent'))

# Replace old FiO2 data with newly merged data - This will overwrite old fio2 values (even those from oxygen delivery table)
data_merged['fio2_original'] = np.where(~data_merged['fio2_vent'].isna(), data_merged['fio2_vent'], data_merged['fio2_original'])
data_merged.drop('fio2_vent', axis = 1, inplace = True)
data_merged.rename(columns = {'fio2_original': 'fio2'}, inplace = True)
data_merged

,time_bucket,stay_id,anchor_age,insurance,race,charlson,marital_status,gender,mortality,deathtime,map,pao2,creatinine,hr,lactate,platelets,epinephrine,norepinephrine,phenylephrine,dobutamine,milrinone,dopamine,ventilation_status,vent,seq_num,o2_delivery_device_1,fio2
0,2160-05-18 10:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,NaN,421.0,NaN,NaN,3.3,NaN,0.0,0.0,0.000,0.0,0.0,0.0,None,0,1,NaN,NaN
1,2160-05-18 11:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,NaN,NaN,NaN,NaN,2.8,NaN,0.0,0.0,0.000,0.0,0.0,0.0,None,0,2,NaN,NaN
2,2160-05-18 12:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,NaN,384.0,NaN,NaN,3.1,NaN,0.0,0.0,0.000,0.0,0.0,0.0,None,0,3,NaN,NaN
3,2160-05-18 13:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,NaN,311.0,NaN,NaN,2.6,252.0,0.0,0.0,0.000,0.0,0.0,0.0,None,0,4,NaN,NaN
4,2160-05-18 14:00:00,39060235,53,Medicare,Other,7,SINGLE,F,False,NaT,94.0,332.0,1.1,80.0,NaN,254.0,0.0,0.0,0.701,0.0,0.0,0.0,InvasiveVent,1,5,NaN,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319026,2185-02-03 10:00:00,34995866,66,Medicare,White,3,DIVORCED,M,False,NaT,NaN,NaN,NaN,104.0,NaN,NaN,0.0,0.0,0.000,0.0,0.0,0.0,SupplementalOxygen,0,20,NaN,NaN
319027,2185-02-03 11:00:00,34995866,66,Medicare,White,3,DIVORCED,M,False,NaT,NaN,NaN,NaN,82.0,NaN,NaN,0.0,0.0,0.000,0.0,0.0,0.0,SupplementalOxygen,0,21,NaN,NaN
319028,2185-02-03 12:00:00,34995866,66,Medicare,White,3,DIVORCED,M,False,NaT,NaN,NaN,NaN,85.0,NaN,NaN,0.0,0.0,0.000,0.0,0.0,0.0,SupplementalOxygen,0,22,Nasal cannula,33.0
319029,2185-02-03 13:00:00,34995866,66,Medicare,White,3,DIVORCED,M,False,NaT,NaN,NaN,NaN,79.0,NaN,NaN,0.0,0.0,0.000,0.0,0.0,0.0,None,0,23,NaN,NaN


Checking correctness of ventilation_status by cross-tabulating with O2 delivery device variable

In [212]:
pd.crosstab(data_merged['ventilation_status'].fillna('Missing'), data_merged['o2_delivery_device_1'].fillna('Missing'), dropna=False, margins = True).T

ventilation_status,HFNC,InvasiveVent,NonInvasiveVent,None,SupplementalOxygen,Tracheostomy,All
o2_delivery_device_1,,,,,,,
Aerosol-cool,1,1355,7,211,3077,18,4669
Bipap mask,0,0,0,1,0,0,1
CPAP mask,0,1,0,0,4,0,5
Face tent,1,508,8,93,1528,0,2138
High flow nasal cannula,824,4,12,39,36,0,915
High flow neb,0,1,0,0,2,0,3
Missing,1528,73384,1454,116195,84854,2260,279675
Nasal cannula,55,1118,102,1527,28544,3,31349
Non-rebreather,2,3,2,11,105,0,123


In [213]:
# Need to recode rows in `ventilation_status` from None to SupplementalOxygen if O2 deliver_device_1 is not missing
data_merged['ventilation_status'] = np.where((data_merged['ventilation_status'] == 'None') & (~data_merged['o2_delivery_device_1'].isna()), 'SupplementalOxygen', data_merged['ventilation_status'])

**Recoding Outlier Values**

In [214]:
# Recoding MAP 
def recode_map(value):
    if value <= 0:
        return np.NaN
    elif value>=220:
        return np.NaN
    else:
        return value
    
data_merged['map'] = data_merged['map'].apply(recode_map)

# Recoding PAO2 
def recode_pao2(value):
    if value <= 0:
        return np.NaN
    elif value>=500:
        return np.NaN
    else:
        return value
    
data_merged['pao2'] = data_merged['pao2'].apply(recode_pao2)

# Recoding creatinine 
def recode_cr(value):
    if value <= 0:
        return np.NaN
    elif value>=999999:
        return np.NaN
    else:
        return value
    
data_merged['creatinine'] = data_merged['creatinine'].apply(recode_cr)

# Recoding lactate 
def recode_lactate(value):
    if value <= 0:
        return np.NaN
    elif value>=5000:
        return np.NaN
    else:
        return value
    
data_merged['lactate'] = data_merged['lactate'].apply(recode_lactate)

# Recoding FiO2 
def recode_fio2(value):
    if value <= 21:
        return 21
    else:
        return value
    
data_merged['fio2'] = data_merged['fio2'].apply(recode_fio2)

# No Need to recode HR

# Recoding Platelet Count & lactic acid (can use the same code as creatinine)
data_merged['platelets'] = data_merged['platelets'].apply(recode_cr)

In [215]:
# Sort by Unique Stay and sequence number to get events in correct order
data_merged.sort_values(by=['stay_id', 'time_bucket', 'seq_num'], inplace=True)

In [216]:
data_merged.head()

,time_bucket,stay_id,anchor_age,insurance,race,charlson,marital_status,gender,mortality,deathtime,map,pao2,creatinine,hr,lactate,platelets,epinephrine,norepinephrine,phenylephrine,dobutamine,milrinone,dopamine,ventilation_status,vent,seq_num,o2_delivery_device_1,fio2
76574,2165-07-31 12:00:00,30004530,63,Medicare,White,5,DIVORCED,M,False,NaT,NaN,305.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,None,0,1,NaN,NaN
76575,2165-07-31 13:00:00,30004530,63,Medicare,White,5,DIVORCED,M,False,NaT,73.333333,300.0,NaN,71.0,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,InvasiveVent,1,2,NaN,100.0
76576,2165-07-31 14:00:00,30004530,63,Medicare,White,5,DIVORCED,M,False,NaT,NaN,NaN,NaN,84.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,InvasiveVent,1,3,NaN,40.0
76577,2165-07-31 15:00:00,30004530,63,Medicare,White,5,DIVORCED,M,False,NaT,NaN,95.0,NaN,83.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,InvasiveVent,1,4,NaN,40.0
76578,2165-07-31 16:00:00,30004530,63,Medicare,White,5,DIVORCED,M,False,NaT,65.000000,96.0,NaN,71.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,InvasiveVent,1,5,NaN,NaN


**Importing GCS Data**

In [217]:
gcs = pd.read_csv('./data/datasets/gcs_cabg.csv')
gcs['time_bucket'] = pd.to_datetime(gcs['charttime']).dt.floor('H')
gcs.drop('charttime', axis = 1, inplace = True)
gcs = gcs.groupby(['stay_id', 'time_bucket']).min().reset_index() # Keeping lowest GCS in an hour

# Merging with main data
data_merged = pd.merge(data_merged, gcs, on = ['stay_id', 'time_bucket'], how = 'left')

**HANDLING MISSING LAB/VITAL SIGNS VALUES**

*Forward/Backward filling of variables*: 

* Average between the preceding and succeeding value. 
* If no precedine value, apply backward filling. 
* If no next/succeeding value, apply forward filling

In [218]:
df = data_merged[['stay_id','map','pao2','creatinine','hr','fio2','lactate','platelets', 'gcs']]
df.head()

,stay_id,map,pao2,creatinine,hr,fio2,lactate,platelets,gcs
0,30004530,NaN,305.0,NaN,NaN,NaN,NaN,NaN,NaN
1,30004530,73.333333,300.0,NaN,71.0,100.0,NaN,NaN,NaN
2,30004530,NaN,NaN,NaN,84.0,40.0,NaN,NaN,15.0
3,30004530,NaN,95.0,NaN,83.0,40.0,NaN,NaN,NaN
4,30004530,65.000000,96.0,NaN,71.0,NaN,NaN,NaN,15.0


In [219]:
def handle_missing_values_optimized(df, id_column='stay_id'):
    """
    Optimized version of missing value handler for lab/vital signs data.
    Uses vectorized operations for better performance.
    
    Parameters:
    -----------
    df : pandas.DataFrame
        Input dataframe containing the time series data
    id_column : str, default='stay_id'
        Column name containing the patient/stay identifier
    
    Returns:
    --------
    pandas.DataFrame
        DataFrame with imputed values
    """
    # Create a copy to avoid modifying the original dataframe
    df_processed = df.copy()
    
    # Get all columns except the ID column
    value_columns = [col for col in df.columns if col != id_column]
    
    # Process each group using vectorized operations
    def process_group_vectorized(group):
        for col in value_columns:
            # Skip if column has no missing values
            if not group[col].isna().any():
                continue
            
            # Get series of non-null values and their indices
            non_null = group[col].dropna()
            
            if len(non_null) == 0:
                continue
                
            # Get indices of missing and non-null values
            missing_idx = group[col].isna()
            
            # Calculate forward and backward fill values
            ffill_values = group[col].ffill()
            bfill_values = group[col].bfill()
            
            # Create arrays for interpolation
            x = np.arange(len(group))
            x_non_null = x[~missing_idx]
            y_non_null = group[col][~missing_idx]
            
            if len(x_non_null) > 1:
                # Use linear interpolation for values between known points
                interpolated = np.interp(
                    x[missing_idx],
                    x_non_null,
                    y_non_null
                )
                
                # Assign interpolated values
                group.loc[missing_idx, col] = interpolated
            else:
                # If only one non-null value exists, use forward/backward fill
                group.loc[missing_idx, col] = np.where(
                    pd.notna(ffill_values[missing_idx]),
                    ffill_values[missing_idx],
                    bfill_values[missing_idx]
                )
        
        return group
    
    # Apply the vectorized processing to each group
    df_processed = df_processed.groupby(id_column, group_keys=False).apply(process_group_vectorized)
    
    return df_processed

# Creating imputed dataset for select variables
df_imputed = handle_missing_values_optimized(df)

# Replace in original dataset
data_imputed = data_merged.copy()
data_imputed[['stay_id','map','pao2','creatinine','hr','fio2','lactate','platelets', 'gcs']] = df_imputed

In [220]:
# Imputing FiO2 values based on ventilation status (fio2 = 21% if no ventilation)
data_imputed['fio2'] = np.where(data_imputed['ventilation_status'] == 'None', 21, data_imputed['fio2'])

# Dropping ventilation variable
data_imputed.drop(['ventilation_status', 'o2_delivery_device_1'], axis = 1, inplace = True)

If patents still have missing fio2 data, even after grouping, averaging, forward, and backward filling, we will recode Fio2 to be equal to 21% if patient is not of ventilator

Spot checking imputed dataset - looking more closely at rows/values that were not imputed

In [221]:
# Missing values for each column
df_imputed.isnull().sum().to_frame().T

,stay_id,map,pao2,creatinine,hr,fio2,lactate,platelets,gcs
0,0,821,430,3425,17,45,14921,1299,819


In [222]:
df_imputed.groupby('stay_id').apply(lambda x: x.isna().all()).sum().to_frame().T

,stay_id,map,pao2,creatinine,hr,fio2,lactate,platelets,gcs
0,0,23,9,119,1,3,368,42,10


In [223]:
# Get the number of patients that have missing values for ALL of the columns map, pao2, creatinine, hr, lactate, and platelet
df_imputed[df_imputed.loc[:, df_imputed.columns != 'stay_id'].isnull().all(axis=1)]['stay_id'].nunique()

0

In [224]:
# Get the number of patients that have missing values for ANY the columns map, pao2, creatinine, hr, lactate, and platelet
df_imputed[df_imputed.loc[:, df_imputed.columns != 'stay_id'].isnull().any(axis=1)]['stay_id'].nunique()

535

In [225]:
df_imputed['stay_id'].nunique()

4451

In [226]:
# Rearranging columns in a way that makes sense
data_imputed = data_imputed[
    [
        'stay_id', 'time_bucket', 'deathtime', 'seq_num',   # Identifiers (add 'subject_id' if available)
        'anchor_age', 'gender', 'race', 'marital_status', 'insurance',  # Demographics
        'vent', 'charlson', # Clinical status
        'map', 'hr', 'pao2', 'fio2', 'creatinine', 'lactate', 'platelets', 'gcs', # Vital signs & lab results
        'epinephrine', 'norepinephrine', 'phenylephrine', 'dobutamine', 'milrinone', 'dopamine',  # Medications
        'mortality' # Outcomes
    ]
]


**CREATING VARIABLE CODING FOR WHETHER PATIENT WAS ALIVE OR NOT AT THE SPECIFIC TIME BUCKET**

In [230]:
data_imputed[~data_imputed.deathtime.isna()].head()

,stay_id,time_bucket,deathtime,seq_num,anchor_age,gender,race,marital_status,insurance,vent,charlson,map,hr,pao2,fio2,creatinine,lactate,platelets,gcs,epinephrine,norepinephrine,phenylephrine,dobutamine,milrinone,dopamine,mortality
8197,30275792,2123-10-08 09:00:00,2123-10-16 19:15:00,1,87,F,White,WIDOWED,Medicare,0,7,80.000000,80.0,406.0,21.0,1.7,1.700000,91.00,15.0,0.0,0.0,0.0,0.0,0.0,0.0,True
8198,30275792,2123-10-08 10:00:00,2123-10-16 19:15:00,2,87,F,White,WIDOWED,Medicare,0,7,80.000000,80.0,375.0,21.0,1.7,1.700000,91.00,15.0,0.0,0.0,0.0,0.0,0.0,0.0,True
8199,30275792,2123-10-08 11:00:00,2123-10-16 19:15:00,3,87,F,White,WIDOWED,Medicare,0,7,80.000000,80.0,349.5,21.0,1.7,1.800000,106.00,15.0,0.0,0.0,1.0,0.0,0.0,0.0,True
8200,30275792,2123-10-08 12:00:00,2123-10-16 19:15:00,4,87,F,White,WIDOWED,Medicare,1,7,140.666667,80.0,324.0,50.0,1.7,1.900000,121.00,15.0,0.0,0.0,1.0,0.0,0.0,0.0,True
8201,30275792,2123-10-08 13:00:00,2123-10-16 19:15:00,5,87,F,White,WIDOWED,Medicare,1,7,78.000000,80.0,312.6,45.0,1.7,1.816667,119.25,15.0,0.0,0.0,1.0,0.0,0.0,0.0,True


In [231]:
# Making sure that all patients who died have a time of death recorded (should return True)
data_imputed[~data_imputed.deathtime.isna()].mortality.value_counts()[0] == data_imputed[~data_imputed.deathtime.isna()].shape[0]

True

In [236]:
data_imputed['status'] = np.where(
    (data_imputed['deathtime'].dt.floor('H') <= data_imputed['time_bucket']) & 
    (data_imputed['mortality'] == True), 
    'dead', 
    'alive'
)

I will create 2 datasets:   
    1. First dataset will exclude patients that have missing values for ANY of the columns map, pao2, creatinine, hr, lactate, and platelet  
    2. The second dataset will impute missing values for these variables with normal physiologic values

In [243]:
# Complete data DF
complete_final_df = data_imputed.copy()
complete_final_df = complete_final_df[~df_imputed.loc[:, df_imputed.columns != 'stay_id'].isnull().any(axis=1)]

# Converting to integers
complete_final_df[['map', 'hr', 'pao2', 'fio2', 'gcs', 'platelets']] = complete_final_df[['map', 'hr', 'pao2', 'fio2', 'gcs', 'platelets']].astype(int)

# Rounding
complete_final_df[['creatinine']] = complete_final_df[['creatinine']].round(2)

complete_final_df

,stay_id,time_bucket,deathtime,seq_num,anchor_age,gender,race,marital_status,insurance,vent,charlson,map,hr,pao2,fio2,creatinine,lactate,platelets,gcs,epinephrine,norepinephrine,phenylephrine,dobutamine,milrinone,dopamine,mortality,status
0,30004530,2165-07-31 12:00:00,NaT,1,63,M,White,DIVORCED,Medicare,0,5,73,71,305,21,1.0,1.3,141,15,0.0,0.0,0.0,0.0,0.0,0.0,False,alive
1,30004530,2165-07-31 13:00:00,NaT,2,63,M,White,DIVORCED,Medicare,1,5,73,71,300,100,1.0,1.3,141,15,0.0,0.0,1.0,0.0,0.0,0.0,False,alive
2,30004530,2165-07-31 14:00:00,NaT,3,63,M,White,DIVORCED,Medicare,1,5,70,84,197,40,1.0,1.3,141,15,0.0,0.0,0.0,0.0,0.0,0.0,False,alive
3,30004530,2165-07-31 15:00:00,NaT,4,63,M,White,DIVORCED,Medicare,1,5,67,83,95,40,1.0,1.3,141,15,0.0,0.0,0.0,0.0,0.0,0.0,False,alive
4,30004530,2165-07-31 16:00:00,NaT,5,63,M,White,DIVORCED,Medicare,1,5,65,71,96,40,1.0,1.3,141,15,0.0,0.0,0.0,0.0,0.0,0.0,False,alive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
319026,39995735,2124-08-27 15:00:00,NaT,261,62,M,White,MARRIED,Other,0,5,63,70,98,21,1.2,0.8,269,15,0.0,0.0,0.0,0.0,0.0,0.0,False,alive
319027,39995735,2124-08-27 16:00:00,NaT,262,62,M,White,MARRIED,Other,0,5,65,70,98,21,1.2,0.8,269,15,0.0,0.0,0.0,0.0,0.0,0.0,False,alive
319028,39995735,2124-08-27 17:00:00,NaT,263,62,M,White,MARRIED,Other,0,5,60,70,98,21,1.2,0.8,269,15,0.0,0.0,0.0,0.0,0.0,0.0,False,alive
319029,39995735,2124-08-27 18:00:00,NaT,264,62,M,White,MARRIED,Other,0,5,60,70,98,21,1.2,0.8,269,15,0.0,0.0,0.0,0.0,0.0,0.0,False,alive


In [244]:
# Imputed data
imputed_final_df = data_imputed.copy()

# Fill missing values with fixed physiologic values
imputed_final_df['map'] = imputed_final_df['map'].fillna(75)
imputed_final_df['hr'] = imputed_final_df['hr'].fillna(70)
imputed_final_df['pao2'] = imputed_final_df['pao2'].fillna(100)
imputed_final_df['creatinine'] = imputed_final_df['creatinine'].fillna(0.8)
imputed_final_df['lactate'] = imputed_final_df['lactate'].fillna(0.7)
imputed_final_df['gcs'] = imputed_final_df['gcs'].fillna(15)
imputed_final_df['platelets'] = imputed_final_df['platelets'].fillna(200)

# Converting to integers
imputed_final_df[['map', 'hr', 'pao2', 'fio2', 'gcs', 'platelets']] = imputed_final_df[['map', 'hr', 'pao2', 'fio2', 'gcs', 'platelets']].astype(int)

# Rounding
imputed_final_df[['creatinine']] = imputed_final_df[['creatinine']].round(2)

In [245]:
print(f"Total number of unique patients in imputed dataset: {imputed_final_df.stay_id.nunique()}, and number of deaths: {imputed_final_df.groupby('stay_id').mortality.any().sum()} ({round((100*imputed_final_df.groupby('stay_id').mortality.any().sum())/imputed_final_df.stay_id.nunique(),2)}%)")
print(f"Total number of unique patients in compelte dataset: {complete_final_df.stay_id.nunique()}, and number of deaths: {complete_final_df.groupby('stay_id').mortality.any().sum()} ({round((100*complete_final_df.groupby('stay_id').mortality.any().sum())/complete_final_df.stay_id.nunique(),2)}%)")

Total number of unique patients in imputed dataset: 4451, and number of deaths: 58 (1.3%)
Total number of unique patients in compelte dataset: 3916, and number of deaths: 57 (1.46%)


In [ ]:
# Saving data to csv
complete_final_df.to_csv('./data/final_data/complete_data.csv', index = True, header=True)
imputed_final_df.to_csv('./data/final_data/imputed_data.csv', index = True, header=True)

# # Imputed data
# for i, chunk in enumerate(range(0, imputed_final_df.shape[0], 100000)):
#     # Creating file names dynamically based on chunk number
#     chunk_df = imputed_final_df.iloc[chunk:chunk + 100000]
#     chunk_df.to_csv(f'../data/final_data/chunked_imputed_data/model_imputed_data_chunk_{i+1}.csv', index=False)

# # Complete non-imputed data
# for i, chunk in enumerate(range(0, complete_final_df.shape[0], 100000)):
#     # Creating file names dynamically based on chunk number
#     chunk_df = complete_final_df.iloc[chunk:chunk + 100000]
#     chunk_df.to_csv(f'../data/final_data/chunked_complete_data/model_complete_data_chunk_{i+1}.csv', index=False)